<a href="https://colab.research.google.com/github/LuigiPagani/BFS-CUDA-for-Graph-Trasversal/blob/main/KTO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Numina 1st Place Solution

Our solution was based on a simple extension to the [self-consistency decoding algorithm](https://arxiv.org/abs/2203.11171) to include tool-integrated reasoning (SC-TIR). This allowed us to generate and prune a diverse set of reasoning traces with code execution from the Python REPL. Concretely, the algorithm works as follows:

<img src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/blog/aimo/sc-tir.png" alt="SC-TIR algorithm" width="800" style="margin-left:'auto' margin-right:'auto' display:'block'"/>

1. For each problem, copy the input $M$ times to define the initial batch of prompts to provide the model. These effectively define the number of candidates one uses for self-consistency / majority voting.
2. Sample $M$ completions until a complete block of Python code is produced (like the DeepSeekMath Instruct/RL models, our model produces code blocks in the ToRA format).
3. Execute each Python block and concatenate the output, including tracebacks if they appear.
4. Repeat $N$ times to produce a set of reasoning traces of width $M$ and depth $N$. If a trace fails to produce sensible outputs (e.g. incomplete code blocks or no `\boxed{}` output) prune that trace.
5. Postprocess the solution candidates and then apply majority voting to select the final answer

To accelerate inference we used [vLLM](https://github.com/vllm-project/vllm) and 8-bit models that were quantized with [AutoGPTQ](https://github.com/AutoGPTQ/AutoGPTQ). On modern hardware, one can skip the quantization step and run inference in standard 16-bit precision.


## Setup and install dependencies

In [1]:
# If using pip
!pip install vllm==0.4.2
!pip install grpcio==1.62.2
!pip install antlr4-python3-runtime==4.11.0
!pip install networkx shapely sage matplotlib gmpy2 scipy numpy sympy mpmath

# If on Kaggle
# !pip uninstall -y torch
# !pip install -U --no-index --find-links=/kaggle/input/vllm-whl -U vllm
# !pip install -U --upgrade /kaggle/input/vllm-t4-fix/grpcio-1.62.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
# !pip install -U --upgrade /kaggle/input/vllm-t4-fix/ray-2.11.0-cp310-cp310-manylinux2014_x86_64.whl
# !pip install -U --upgrade /kaggle/input/antlr4-python3-runtime-package-4-11/antlr4_python3_runtime-4.11.0-py3-none-any.whl

  Preparing metadata (setup.py) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nv

In [2]:
!pip install networkx shapely sage matplotlib gmpy2 scipy numpy sympy mpmath


ERROR: Ignored the following yanked versions: 0.0.0
ERROR: Could not find a version that satisfies the requirement sage (from versions: none)
ERROR: No matching distribution found for sage


In [3]:
!pip install accelerate==0.28.0 \
            auto_gptq==0.7.1 \
            datasets==2.18.0 \
            huggingface_hub==0.23.4 \
            "numpy<2.0.0" \
            tensorboard==2.17.0 \
            transformers==4.41.2 \
            trl==0.8.1 \
            deepspeed==0.14.0 \
            wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 62.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 42.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 402.6/402.6 kB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 116.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.0/225.0 kB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 123.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.9/170.9 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 22.9 MB/s eta 0:00:0

## Imports

In [4]:
import os
import re
import signal
import subprocess
import tempfile
from collections import Counter
from contextlib import contextmanager
from dataclasses import dataclass

import pandas as pd
from datasets import load_dataset, Dataset, concatenate_datasets
import torch
from transformers import set_seed
from tqdm import tqdm
from vllm import LLM, SamplingParams

## Configuration

We found it useful to define a single `Config` class that gathers all the setting used for a single submission:

In [5]:
@dataclass
class Config:
    model_id: str

    # Decoding Parameters
    num_samples: int        # Number of candidates to generate (width)
    num_generations: int    # Number of steps to generate per candidate (depth)
    restart_on_fail: bool   # Regenerate a step if it fails to generate Python codeblocks

    # Sampling Parameters
    temperature: float
    max_new_tokens: int

    # Runtime Parameters
    validation_set: str     # One of AI-MO/aimo-validation-amc, AI-MO/aimo-validation-aime, AI-MO/aimo-validation-math-level-4, AI-MO/aimo-validation-math-level-5
    is_submission: bool = bool(os.getenv("KAGGLE_IS_COMPETITION_RERUN"))

## Task environment setup

In [6]:
def get_kaggle_env(config):
    """Adapted from: https://www.kaggle.com/code/eabdullin/mathgenie-interlm-20b-interactive-code-running"""
    if config.is_submission:
        import aimo

        env = aimo.make_env()
        iter_test = env.iter_test()
        return env, iter_test

    def get_train_data():
        dataset = load_dataset(config.validation_set, split="train[:10]") # replace with `train` to evaluate over the full validation set
        dataset = dataset.map(lambda x: {'answer': str(int(x['answer']) % 1000)})
        df = dataset.to_pandas()
        return df

    class train_env:
        def __init__(self, shuffle=False):
            self.shuffle = shuffle
            self.df = get_train_data()
            self.df["ground_truth"] = self.df["answer"]
            self.df["answer"] = -1
            if self.shuffle:
                self.df = self.df.reset_index().sample(frac=1).reset_index(drop=True)
            self.predict_called = True
            self.counter = 0
            self.len = len(self.df)

        def iter_test(self):
            while self.counter < self.len:
                if self.predict_called:
                    self.predict_called = False
                    yield (self.df.loc[[self.counter]][["id", "problem"]]), (self.df.loc[[self.counter]][["id", "answer"]])
                else:
                    print("You must call `predict()` successfully before you can continue with `iter_test()`")
                    yield None

        def predict(self, answer):
            self.df[self.counter, "answer"] = answer["answer"]
            self.predict_called = True
            self.counter += 1

    env = train_env(shuffle=True)
    iter_test = env.iter_test()

    return env, iter_test

## vLLM and model generation utilities

In [7]:
def build_vllm(config):
    num_gpus = torch.cuda.device_count()
    if "awq" in config.model_id.lower():
        quantization = "AWQ"
    elif "gptq" in config.model_id.lower():
        quantization = "gptq"
    else:
        quantization = None
    vllm = LLM(
        model=config.model_id,
        tensor_parallel_size=num_gpus,
        quantization=quantization,
        swap_space=0,
    )
    return vllm


def apply_template(sample, tokenizer, prompt):
    messages = [{"role": "user", "content": prompt.format(sample["prompt"], "{}")}]
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    sample["text"] = text
    return sample


def generate_batched(samples, vllm, sampling_params):
    outputs = vllm.generate(samples["gen_texts"], sampling_params, use_tqdm=True)
    samples["gen_texts"] = [o.prompt + o.outputs[0].text for o in outputs]
    return samples

## Python REPL and code execution utilities

In [8]:
class PythonREPL:
    def __init__(self, timeout=5):
        self.timeout = timeout

    @contextmanager
    def time_limit(self, seconds):
        def signal_handler(*_):
            raise TimeoutError(f"Timed out after {seconds} seconds.")

        signal.signal(signal.SIGALRM, signal_handler)
        signal.alarm(seconds)
        try:
            yield
        finally:
            signal.alarm(0)

    def __call__(self, query):
        query = "import math\nimport numpy as np\nimport sympy as sp\n" + query
        query = query.strip().split("\n")
        if "print(" not in query[-1]:
            if "#" in query[-1]:
                query[-1] = query[-1].split("#")[0]
            query[-1] = "print(" + query[-1] + ")"
        query = "\n".join(query)
        with tempfile.TemporaryDirectory() as temp_dir:
            temp_file_path = os.path.join(temp_dir, "tmp.py")
            with open(temp_file_path, "w", encoding="utf-8") as f:
                f.write(query)
            with self.time_limit(self.timeout):
                result = subprocess.run(
                    ["python3", temp_file_path],
                    capture_output=True,
                    check=False,
                    text=True,
                    timeout=self.timeout,
                )
                if result.returncode == 0:
                    output = result.stdout
                    return True, output.strip()
                error_msg = result.stderr.strip()
                msgs = error_msg.split("\n")
                new_msgs = []
                want_next = False
                for m in msgs:
                    if "Traceback" in m:
                        new_msgs.append(m)
                    elif m == msgs[-1]:
                        new_msgs.append(m)
                    elif temp_file_path in m:
                        st = m.index('"/') + 1 if '"/' in m else 0
                        ed = m.index(temp_file_path) + 1 if temp_file_path in m else None
                        clr = m[st:ed] if not ed else m[st:]
                        m = m.replace(clr, "")
                        new_msgs.append(m)
                        want_next = True
                    elif want_next:
                        new_msgs.append(m)
                        want_next = False
                error_msg = "\n".join(new_msgs)
                return False, error_msg.strip()


def execute_completion(executor, completion, return_status, last_code_block):
    executions = re.findall(r"```python(.*?)```", completion, re.DOTALL)
    if len(executions) == 0:
        return completion, False if return_status else completion
    if last_code_block:
        executions = [executions[-1]]
    outputs = []
    successes = []
    for code in executions:
        success = False
        for lib in ("subprocess", "venv"):
            if lib in code:
                output = f"{lib} is not allowed"
                outputs.append(output)
                successes.append(success)
                continue
        try:
            success, output = executor(code)
        except TimeoutError as e:
            print("Code timed out")
            output = e
        if not success and not return_status:
            output = ""
        outputs.append(output)
        successes.append(success)
    output = str(outputs[-1]).strip()
    success = successes[-1]
    if return_status:
        return output, success
    return output


def postprocess_completion(text, return_status, last_code_block):
    executor = PythonREPL()
    result = execute_completion(executor, text, return_status=return_status, last_code_block=last_code_block)
    del executor
    return result

## Post-processing and solution extraction utilities

In [9]:
def extract_boxed_answer(text):
    def last_boxed_only_string(text):
        idx = text.rfind("\\boxed")
        if idx < 0:
            idx = text.rfind("\\fbox")
            if idx < 0:
                return None
        i = idx
        right_brace_idx = None
        num_left_braces_open = 0
        while i < len(text):
            if text[i] == "{":
                num_left_braces_open += 1
            if text[i] == "}":
                num_left_braces_open -= 1
                if num_left_braces_open == 0:
                    right_brace_idx = i
                    break
            i += 1
        if right_brace_idx is None:
            return None
        return text[idx : right_brace_idx + 1]

    def remove_boxed(boxed):
        left = "\\boxed{"
        try:
            assert boxed[: len(left)] == left
            assert boxed[-1] == "}"
            length = len(left)
            return boxed[length:-1]
        except Exception:
            return None

    boxed = last_boxed_only_string(text)
    if boxed is None:
        return None
    answer = remove_boxed(boxed)
    return answer


def normalize_answer(answer):
    match = re.search(r"(.*?)Problem:", answer, flags=re.S)
    if match:
        answer = match.group(1)
    subs = [("an ", ""), ("a ", ""), (".$", "$"), ("\\$", ""), (r"\ ", ""), (" ", ""), ("mbox", "text"), (",\\text{and}", ","), ("\\text{and}", ","), ("\\text{m}", "\\text{}"), ("\\le", "<")]
    remove = ["square", "ways", "integers", "dollars", "mph", "inches", "ft", "hours", "km", "units", "\\ldots", "sue", "points", "feet", "minutes", "digits", "cents", "degrees", "cm", "gm", "pounds", "meters", "meals", "edges", "students", "childrentickets", "multiples", "\\text{s}", "\\text{.}", "\\text{\ns}", "\\text{}^2", "\\text{}^3", "\\text{\n}", "\\text{}", r"\mathrm{th}", r"^\circ", r"^{\circ}", r"\;", r",\!", "{,}", '"', "\\dots", "\n", "\r", "\f", "\%"]
    sub_patterns = [r"(\\text\{)(.*?)(\})", r"(\\textbf\{)(.*?)(\})", r"(\\overline\{)(.*?)(\})", r"(\\boxed\{)(.*)(\})"]
    split_patterns = [r"finalansweris(.*)", r"answer?is:?(.*)", r"oxed\{(.*?)\}", r"\$(.*?)\$"]
    for before, after in subs:
        answer = answer.replace(before, after)
    for expr in remove:
        answer = answer.replace(expr, "")
    for pattern in sub_patterns:
        answer = re.sub(pattern, "\\2", answer)
    for pattern in split_patterns:
        if len(re.findall(pattern, answer)) > 0:
            answer = re.findall(pattern, answer)[-1]
    answer = answer.strip()
    if "rac" in answer and "\\frac" not in answer:
        answer = answer.replace("rac", "\\frac")
    answer = re.sub(r"(frac)([^{])(.)", "frac{\\2}{\\3}", answer)
    answer = re.sub(r"(sqrt)([^{])", "sqrt{\\2}", answer)
    answer = answer.replace("$", "")
    if answer.replace(",", "").isdigit():
        answer = answer.replace(",", "")
    return answer

## SC-TIR control flow

In [10]:
def process_code(sample, restart_on_fail, last_step, check_last_n_chars=100):
    gen_text = sample["gen_texts"]
    num_python_blocks = len(re.findall(r"```python(.*?)```", gen_text, re.DOTALL))
    region_to_check = gen_text[-check_last_n_chars:]
    if num_python_blocks == 0:
        if restart_on_fail:
            print("no code has ever been generated, RESTARTING")
            sample["gen_texts"] = sample["text"]
        else:
            print("no code has ever been generated, STOP")
            sample["should_prune"] = True
            sample["has_code"] = False
        return sample
    if not gen_text.endswith("```output\n") and ("answer is" in region_to_check or "\\boxed" in region_to_check):
        num_output_blocks = len(re.findall(r"```output(.*?)```", gen_text, re.DOTALL))
        if num_output_blocks == 0:
            print("The model hallucinated the code answer")
            sample["should_prune"] = True
            return sample
        if "boxed" in region_to_check:
            try:
                answer = normalize_answer(extract_boxed_answer(region_to_check))
            except Exception:
                answer = "-1"
        else:
            answer = normalize_answer(region_to_check)
        sample["model_answers"] = answer
        return sample
    if last_step:
        return sample
    if not gen_text.endswith("```output\n"):
        print("warning: output block not found: ", gen_text[-40:])
        if restart_on_fail:
            sample["gen_texts"] = sample["text"]
        else:
            sample["should_prune"] = True
        return sample
    code_result, _ = postprocess_completion(gen_text, return_status=True, last_code_block=True)
    truncation_limit = 200
    if len(code_result) > truncation_limit:
        code_result = code_result[:truncation_limit] + " ... (output truncated)"
    sample["gen_texts"] = gen_text + f"{code_result}\n```"
    return sample

## Sample filtering and majority voting

In [11]:
def filter_answers(answers):
    def validate_answer_is_numeric(x, tolerance=0.2):
        try:
            x = round(float(x))
            f = float(x)
            if abs(x - f) > tolerance:
                x = -1
        except Exception:
            x = -1
        return x

    formatted = [validate_answer_is_numeric(a) for a in answers]
    filtered = [a % 1000 for a in formatted if a >= 0]
    return filtered


def get_majority_vote(answers):
    if not len(answers):
        return 0
    c = Counter(answers)
    value, _ = c.most_common()[0]
    return value

**Main Mine**

In [ ]:
import numpy as np

class KTOChainOfThought:
    def __init__(self, config, vllm):
        self.config = config
        self.vllm = vllm

    def compute_kto_loss(self, r_theta, z0, is_desirable):
        beta = self.config.beta
        lambda_d = self.config.lambda_d
        lambda_u = self.config.lambda_u
        z = r_theta - z0
        if is_desirable:
            return lambda_d * (1 - 1 / (1 + np.exp(-beta * z)))
        else:
            return lambda_u * (1 / (1 + np.exp(-beta * z)))

    def solve(self, samples):
        candidates = samples["model_answers"]
        filtered = filter_answers(candidates)

        if not filtered:
            return 0

        z0 = np.mean([np.log(len(c)) for c in samples["gen_texts"]])
        losses = []
        for completion, answer in zip(samples["gen_texts"], samples["model_answers"]):
            r_theta = np.log(len(completion))
            is_desirable = answer != "-1"
            loss = self.compute_kto_loss(r_theta, z0, is_desirable)
            losses.append((loss, int(answer) if answer.isdigit() else -1))

        best_answer = min(losses, key=lambda x: x[0])[1]
        return best_answer % 1000

def main_kto(config):
    print(f"=== Running KTO submission with config ===\n\n{config}")
    set_seed(42)
    num_procs = os.cpu_count()
    vllm = build_vllm(config)
    kto = KTOChainOfThought(config, vllm)
    sampling_params = SamplingParams(
        temperature=config.temperature,
        max_tokens=config.max_new_tokens,
        stop=["```output\n"],
        include_stop_str_in_output=True,
    )
    env, iter_test = get_kaggle_env(config)
    final_answers = []
    for test, submission in tqdm(iter_test, desc="Solving problems"):
        problem = apply_template({"prompt": test.problem.values[0]}, tokenizer=vllm.get_tokenizer(), prompt="{}")
        print(f"=== INPUT FOR PROBLEM ID {test.id.values[0]} ===\n{problem}\n")
        samples = Dataset.from_list([
            {
                "text": problem["text"],
                "gen_texts": problem["text"],
                "should_prune": False,
                "model_answers": "-1",
                "has_code": True,
            }
            for _ in range(config.num_samples)
        ])
        completed = []
        for step in range(config.num_generations):
            samples = samples.map(
                generate_batched,
                batch_size=128,
                batched=True,
                fn_kwargs={"vllm": vllm, "sampling_params": sampling_params},
                load_from_cache_file=False,
            )
            samples = samples.map(
                process_code,
                num_proc=num_procs,
                load_from_cache_file=False,
                fn_kwargs={"restart_on_fail": config.restart_on_fail, "last_step": step == (config.num_generations - 1)},
            )
            done = samples.filter(lambda x: x["should_prune"] is True, load_from_cache_file=False)
            if len(done):
                completed.append(done)
            samples = samples.filter(lambda x: x["should_prune"] is False, load_from_cache_file=False)
        completed.append(samples)
        samples = concatenate_datasets(completed)

        majority = kto.solve(samples)
        print(f"=== KTO ANSWER (mod 1000) ===\n{majority}\n")

        submission["answer"] = majority
        env.predict(submission)
        test["model_answer"] = majority
        final_answers.append(test)
    if not config.is_submission:
        answers = env.df.merge(pd.concat(final_answers))
        answers["correct"] = answers["ground_truth"].astype(int) == answers["model_answer"].astype(int)
        print("Accuracy", answers["correct"].astype(int).mean())

# Update the Config class to include KTO-specific parameters
@dataclass
class Config:
    model_id: str
    num_samples: int
    num_generations: int
    restart_on_fail: bool
    temperature: float
    max_new_tokens: int
    validation_set: str
    is_submission: bool = bool(os.getenv("KAGGLE_IS_COMPETITION_RERUN"))
    beta: float = 0.1
    lambda_d: float = 1.0
    lambda_u: float = 1.0

# Run KTO
config = Config(
    model_id="AI-MO/NuminaMath-7B-TIR-GPTQ",
    num_samples=48,
    num_generations=4,
    restart_on_fail=True,
    temperature=0.8,
    max_new_tokens=2048,
    validation_set="AI-MO/aimo-validation-amc",
    beta=0.1,
    lambda_d=1.0,
    lambda_u=1.0,
)
main_kto(config)

=== Running KTO submission with config ===

Config(model_id='AI-MO/NuminaMath-7B-TIR-GPTQ', num_samples=48, num_generations=4, restart_on_fail=True, temperature=0.8, max_new_tokens=2048, validation_set='AI-MO/aimo-validation-amc', is_submission=False, beta=0.1, lambda_d=1.0, lambda_u=1.0)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.04k [00:00<?, ?B/s]

WARNING 08-25 20:09:52 config.py:172] The model is convertible to Marlin format, but you specified quantization=gptq. Use quantization=marlin for faster inference.
WARNING 08-25 20:09:52 config.py:205] gptq quantization is not fully optimized yet. The speed can be slower than non-quantized models.
INFO 08-25 20:09:52 llm_engine.py:100] Initializing an LLM engine (v0.4.2) with config: model='AI-MO/NuminaMath-7B-TIR-GPTQ', speculative_config=None, tokenizer='AI-MO/NuminaMath-7B-TIR-GPTQ', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=4096, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, disable_custom_all_reduce=False, quantization=gptq, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), seed=0, served_model_name=AI-MO/NuminaMath-7B-TIR-GPTQ)


tokenizer_config.json:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

...

Code timed out


Filter:   0%|          | 0/48 [00:00<?, ? examples/s]

Filter:   0%|          | 0/48 [00:00<?, ? examples/s]

Map:   0%|          | 0/48 [00:00<?, ? examples/s]



Processed prompts:   0%|          | 0/48 [00:00<?, ?it/s]

Processed prompts:   2%|▏         | 1/48 [00:02<01:41,  2.15s/it]

Processed prompts:   4%|▍         | 2/48 [00:03<01:13,  1.59s/it]

Processed prompts:   8%|▊         | 4/48 [00:04<00:43,  1.01it/s]

Processed prompts:  12%|█▎        | 6/48 [00:05<00:34,  1.23it/s]

Processed prompts:  15%|█▍        | 7/48 [00:07<00:37,  1.09it/s]

Processed prompts:  21%|██        | 10/48 [00:08<00:24,  1.54it/s]

Processed prompts:  25%|██▌       | 12/48 [00:09<00:21,  1.68it/s]



## Main loop

In [ ]:
def main(config):
    print(f"=== Running submission with config ===\n\n{config}")
    set_seed(42)
    num_procs = os.cpu_count()
    vllm = build_vllm(config)
    sampling_params = SamplingParams(
        temperature=config.temperature,
        max_tokens=config.max_new_tokens,
        stop=["```output\n"],
        include_stop_str_in_output=True,
    )
    env, iter_test = get_kaggle_env(config)
    final_answers = []
    for test, submission in tqdm(iter_test, desc="Solving problems"):
        problem = apply_template({"prompt": test.problem.values[0]}, tokenizer=vllm.get_tokenizer(), prompt="{}")
        print(f"=== INPUT FOR PROBLEM ID {test.id.values[0]} ===\n{problem}\n")
        samples = Dataset.from_list([
            {
                "text": problem["text"],
                "gen_texts": problem["text"],
                "should_prune": False,
                "model_answers": "-1",
                "has_code": True,
            }
            for _ in range(config.num_samples)
        ])
        completed = []
        for step in range(config.num_generations):
            samples = samples.map(
                generate_batched,
                batch_size=128,
                batched=True,
                fn_kwargs={"vllm": vllm, "sampling_params": sampling_params},
                load_from_cache_file=False,
            )
            samples = samples.map(
                process_code,
                num_proc=num_procs,
                load_from_cache_file=False,
                fn_kwargs={"restart_on_fail": config.restart_on_fail, "last_step": step == (config.num_generations - 1)},
            )
            done = samples.filter(lambda x: x["should_prune"] is True, load_from_cache_file=False)
            if len(done):
                completed.append(done)
            samples = samples.filter(lambda x: x["should_prune"] is False, load_from_cache_file=False)
        completed.append(samples)
        samples = concatenate_datasets(completed)
        candidates = samples["model_answers"]
        print(f"=== CANDIDATE ANSWERS ({len(candidates)}) ===\n{candidates}\n")
        filtered = filter_answers(candidates)
        print(f"=== FILTERED ANSWERS ({len(filtered)}) ===\n{filtered}\n")
        majority = get_majority_vote(filtered)
        print(f"=== MAJORITY ANSWER (mod 1000) ===\n{majority}\n")
        submission["answer"] = majority
        env.predict(submission)
        test["model_answer"] = majority
        final_answers.append(test)
    if not config.is_submission:
        answers = env.df.merge(pd.concat(final_answers))
        answers["correct"] = answers["ground_truth"].astype(int) == answers["model_answer"].astype(int)
        print("Accuracy", answers["correct"].astype(int).mean())

## Specify config and run

In [ ]:
config = Config(
    model_id = "AI-MO/NuminaMath-7B-TIR-GPTQ",
    num_samples=10,
    num_generations=4,
    restart_on_fail=True,
    temperature=0.8,
    max_new_tokens=2048,
    validation_set="AI-MO/aimo-validation-amc",
)
main(config)



=== Running submission with config ===

Config(model_id='AI-MO/NuminaMath-7B-TIR-GPTQ', num_samples=10, num_generations=4, restart_on_fail=True, temperature=0.8, max_new_tokens=2048, validation_set='AI-MO/aimo-validation-amc', is_submission=False)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


WARNING 08-25 19:24:08 config.py:172] The model is convertible to Marlin format, but you specified quantization=gptq. Use quantization=marlin for faster inference.
WARNING 08-25 19:24:08 config.py:205] gptq quantization is not fully optimized yet. The speed can be slower than non-quantized models.
INFO 08-25 19:24:08 llm_engine.py:100] Initializing an LLM engine (v0.4.2) with config: model='AI-MO/NuminaMath-7B-TIR-GPTQ', speculative_config=None, tokenizer='AI-MO/NuminaMath-7B-TIR-GPTQ', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=4096, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, disable_custom_all_reduce=False, quantization=gptq, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), seed=0, served_model_name=AI-MO/NuminaMath-7B-TIR-GPTQ)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


INFO 08-25 19:24:09 utils.py:660] Found nccl from library /root/.config/vllm/nccl/cu12/libnccl.so.2.18.1
INFO 08-25 19:24:10 selector.py:81] Cannot use FlashAttention-2 backend because the flash_attn package is not found. Please install it for better performance.
INFO 08-25 19:24:10 selector.py:32] Using XFormers backend.
INFO 08-25 19:24:12 weight_utils.py:199] Using model weights format ['*.safetensors']
INFO 08-25 19:24:15 model_runner.py:175] Loading model weights took 7.3827 GB
INFO 08-25 19:24:18 gpu_executor.py:114] # GPU blocks: 1446, # CPU blocks: 0
INFO 08-25 19:24:18 model_runner.py:937] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 08-25 19:24:18 model_runner.py:941] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager

...

Map (num_proc=10):   0%|          | 0/10 [00:00<?, ? examples/s]

Filter:   0%|          | 0/10 [00:00<?, ? examples/s]

Filter:   0%|          | 0/10 [00:00<?, ? examples/s]

Solving problems: 10it [20:57, 125.77s/it]

=== CANDIDATE ANSWERS (10) ===
['2', '2', '2', '2', '2', '2', '2', '2', '2', 'verycloseto2.']

=== FILTERED ANSWERS (9) ===
[2, 2, 2, 2, 2, 2, 2, 2, 2]

=== MAJORITY ANSWER (mod 1000) ===
2

Accuracy 0.7
